In [1]:
#Importing all required libraries
import pandas as pd
import numpy as np
import time
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
from sqlalchemy import create_engine


In [2]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [3]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'kevin' 
POSTGRES_DBNAME = 'postgres'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))
# Create the connection
try:
    print('Connecting to the PostgreSQL...........')
    cnx = create_engine(postgres_str)
    print("Connection successfully..................")
except OperationalError as err:
    # passing exception to function
    show_psycopg2_exception(err)        
    # set the connection to 'None' in case of error

Connecting to the PostgreSQL...........
Connection successfully..................


In [5]:
#Establishing the connection
conn = psycopg2.connect(
   database=POSTGRES_DBNAME, user=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, host=POSTGRES_ADDRESS, port= POSTGRES_PORT
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Droping VISITS table if already exists.
cursor.execute("DROP TABLE IF EXISTS VISITS")

#Creating table as per requirement
sql ='''CREATE TABLE VISITS(
    Customer_id CHAR(20),
    City_id_visited INT,
    Date_visited date
)'''
cursor.execute(sql)
print("VISITS table created successfully........")
conn.commit()

VISITS table created successfully........


In [7]:
#Droping CUSTOMER table if already exists.
cursor.execute("DROP TABLE IF EXISTS CUSTOMER")

#Creating table as per requirement
sql ='''CREATE TABLE CUSTOMER(
    Customer_id CHAR(20),
    Customer_name CHAR(40),
    Gender CHAR(1),
    Age INT
)'''
cursor.execute(sql)
print("CUSTOMER table created successfully........")
conn.commit()

CUSTOMER table created successfully........


In [8]:
#Droping CITY table if already exists.
cursor.execute("DROP TABLE IF EXISTS CITY")

#Creating table as per requirement
sql ='''CREATE TABLE CITY(
    City_id INT,
    City_name CHAR(20),
    Expense INT
)'''
cursor.execute(sql)
print("CITY table created successfully........")
conn.commit()

CITY table created successfully........


In [9]:
#Establishing the connection
conn = psycopg2.connect(
   database=POSTGRES_DBNAME, user=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, host=POSTGRES_ADDRESS, port= POSTGRES_PORT
)
cursor = conn.cursor()
#Inserting sample data into the tables
sql = '''INSERT INTO VISITS (Customer_id,City_id_visited,Date_visited) VALUES
    (1001,2003,'1-Jan-03'),
    (1001,2004,'1-Jan-04'),
    (1002,2001,'1-Jan-01'),
    (1004,2003,'1-Jan-03')'''
cursor.execute(sql)
print("Data inserted in VISITS")
conn.commit()
sql = '''INSERT INTO CUSTOMER (Customer_id,Customer_name,Gender,Age) VALUES
    (1001,'John','M',25),
    (1002,'Mark','M',40),
    (1003,'Martha','F',55),
    (1004,'Selena','F',34)'''
cursor.execute(sql)
print("Data inserted in CUSTOMER")
conn.commit()
sql = '''INSERT INTO CITY (City_id,City_name,Expense) VALUES
    (2001,'Chicago',500),
    (2002,'Newyork',1000),
    (2003,'SFO',2000),
    (2004,'Florida',800)'''
cursor.execute(sql)
print("Data inserted in CITY")
conn.commit()

Data inserted in VISITS
Data inserted in CUSTOMER
Data inserted in CITY


In [10]:
#1) Cities frequently visited?

sql='''select v.city_id_visited,
            trim(c.city_name) as city_name,
            count(distinct customer_id) as count 
            from visits v 
            join city c on c.city_id=v.city_id_visited 
            group by v.city_id_visited,c.city_name 
            having count(distinct customer_id)>1'''
cursor.execute(sql)
output1=cursor.fetchall()
print("Cities with more than 1 visits:")
for row in output1:
    print(row[1])

Cities with more than 1 visits:
SFO


In [11]:
#2) Customers visited more than 1 city?

sql='''select v.customer_id,
            trim(c.customer_name) as customer_name,
            count(distinct city_id_visited) as count 
            from visits v 
            join customer c on c.customer_id=v.customer_id 
            group by v.customer_id,c.customer_name 
            having count(distinct city_id_visited)>1'''
cursor.execute(sql)
output1=cursor.fetchall()
print("Customers that visited more than 1 city:")
for row in output1:
    print(row[1])

Customers that visited more than 1 city:
John


In [12]:
#3) Cities visited breakdown by gender?

sql='''select city_id,trim(city_name) as city_name,gender,count(*) as count
from visits v
join customer cust on v.customer_id=cust.customer_id
join city on city.city_id=v.city_id_visited
group by city_id,city_name,gender'''
cursor.execute(sql)
output1=cursor.fetchall()
print(output1)

[(2003, 'SFO', 'F', 1), (2001, 'Chicago', 'M', 1), (2004, 'Florida', 'M', 1), (2003, 'SFO', 'M', 1)]


In [13]:
#4) List the city names that are not visited by every customer and order them by the expense budget in ascending order?

sql='''select trim(city_name) as city_name, expense from city join (select city_id,count(distinct cust.customer_id) 
from city 
left join visits v on city.city_id=v.city_id_visited
left join customer cust on v.customer_id=cust.customer_id
group by city_id having count(distinct cust.customer_id)<(select count(distinct customer_id) from customer))temp_c on city.city_id=temp_c.city_id order by expense asc'''
cursor.execute(sql)
output1=cursor.fetchall()
for row in output1:
    print("Cities not visited by every customer: ",row[0])
    print("Expense: ",row[1])

Cities not visited by every customer:  Chicago
Expense:  500
Cities not visited by every customer:  Florida
Expense:  800
Cities not visited by every customer:  Newyork
Expense:  1000
Cities not visited by every customer:  SFO
Expense:  2000


In [14]:
#5) Visit/travel Percentage for every customer?

sql='''select distinct trim(cust.customer_name) as customer_name,(count(c.city_id) over (partition by cust.customer_id)*100/count(c.city_id) over ()) as travel_perc from customer cust
left join visits v on v.customer_id=cust.customer_id
left join city c on v.city_id_visited=c.city_id '''
cursor.execute(sql)
output=cursor.fetchall()
for row in output:
    print("Customer: ",row[0])
    print("Travel %: ",row[1])

Customer:  Martha
Travel %:  0
Customer:  John
Travel %:  50
Customer:  Selena
Travel %:  25
Customer:  Mark
Travel %:  25


In [15]:
#6) Total expense incurred by customers on their visits?

sql ='''select trim(cust.customer_id) as customer_id,
trim(cust.customer_name) as customer_name,
case when sum(c.expense) is null then 0 else sum(c.expense) end as expense 
from customer cust 
left join visits v on v.customer_id=cust.customer_id 
left join city c on v.city_id_visited=c.city_id 
group by cust.customer_id,cust.customer_name'''
cursor.execute(sql)
output=cursor.fetchall()
for row in output:
    print("Customer: ",row[1])
    print("Expense: ",row[2])

Customer:  John
Expense:  2800
Customer:  Mark
Expense:  500
Customer:  Martha
Expense:  0
Customer:  Selena
Expense:  2000


In [16]:
#7) list the Customer details along with the city they first visited and the date of visit?

sql='''select trim(customer_id) as customer_id,trim(customer_name) as customer_name,gender,age,trim(city_name) as city_name,to_char(date_visited,'yyyy-mm-dd') as date_visited 
from (select cust.*,trim(c.city_name) as city_name,rank() over (partition by cust.customer_id order by v.date_visited asc) as rnk,v.date_visited from customer cust 
join visits v on v.customer_id=cust.customer_id
join city c on v.city_id_visited=c.city_id)a where rnk=1
'''
cursor.execute(sql)
output=cursor.fetchall()
print(output)

[('1001', 'John', 'M', 25, 'SFO', '2003-01-01'), ('1002', 'Mark', 'M', 40, 'Chicago', '2001-01-01'), ('1004', 'Selena', 'F', 34, 'SFO', '2003-01-01')]
